# Prediction

Apply selected model in full `training` data (cleaned version of `train_features.csv`)

In [1]:
import utils
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb

## Load data

In [2]:
train = pd.read_pickle("training.pkl")
test = pd.read_pickle("testing.pkl")
y_train = train['salary']
jobId = test['jobId']

In [3]:
vars_drop = ['major_new', 'jobId', 'salary']
X_train, _, _ = utils.drop_vars(vars_drop, train)  
categorical_vars = utils.get_categorical_variables(X_train)

In [4]:
vars_drop = ['major_new', 'jobId']
X_test, _, _ = utils.drop_vars(vars_drop, test )  

In [5]:
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in categorical_vars:
    le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])
for col in categorical_vars:
    le.fit(X_test[col])
    X_test[col] = le.transform(X_test[col])

In [6]:
X_test.head()

,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,5,2,7,3,22,73
1,4,4,7,0,20,47
2,2,3,0,3,17,9
3,5,2,7,4,14,96
4,4,1,0,4,10,44


In [7]:
best_model = lgb.Booster(model_file='GBM_label.txt')
best_model.params['objective'] = 'regression'
best_model.refit(X_train, y_train)
train_pred = best_model.predict(X_train)
utils.check_RMSE('refit', y_train, train_pred)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017093 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 154
[LightGBM] [Info] Number of data points in the train set: 999692, number of used features: 6
RMSE of model refit is:  18.77438193278289


In [8]:
gbm_pred = best_model.predict(X_test)

In [9]:
test_salary = pd.DataFrame({'jobId': list(jobId), 'salary': gbm_pred})

## Save results

In [11]:
test_salary.to_csv('test_salaries.csv', index=False)